# Zero-shot object detection

We show an example of running zero-shot object detection, namely Google's OWL-ViT model ([HF model link](https://huggingface.co/google/owlvit-base-patch32). In this example, we will also quickly externalize into a Gradio app. This notebook is tested on Pytorch 2.0.0 Python 3.10 GPU image on SageMaker Studio

In [ ]:
!pip install transformers

## Model setup
We pull the model from HuggingFace Model Hub, and show some examples

In [ ]:
from transformers import pipeline
checkpoint = "google/owlvit-base-patch32"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [ ]:
import numpy as np
from PIL import Image
image = Image.open('meme.jpg').convert("RGB")

In [ ]:
image

In [ ]:
predictions = detector(
    image,
    candidate_labels=["human face", "plaid shirt"],
    threshold=0.05,
)

In [ ]:
predictions

In [ ]:
from PIL import ImageDraw
draw = ImageDraw.Draw(image)

for prediction in predictions:
    box = prediction["box"]
    label = prediction["label"]
    score = prediction["score"]

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="white")

In [ ]:
image

In [ ]:
import numpy as np
from PIL import Image
image = Image.open('keanu.jpeg').convert("RGB")

In [ ]:
image

In [ ]:
predictions = detector(
    image,
    candidate_labels=["jeans", "blazer", "human face", "food"],
    threshold=0.05,
)

In [ ]:
predictions

In [ ]:
from PIL import ImageDraw
draw = ImageDraw.Draw(image)

for prediction in predictions:
    box = prediction["box"]
    label = prediction["label"]
    score = prediction["score"]

    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="white")

In [ ]:
image

## Quick externalization for demonstration

Gradio provides a quick way to externalize your demo to the outside world. Here's a simple example

In [ ]:
!pip install gradio  --upgrade

In [ ]:
import gradio as gr
def detect_zeroshot(labels,image):
    predictions = detector(
        image,
        candidate_labels=labels.split(','),
        threshold=0.05,
    )
    draw = ImageDraw.Draw(image)
    for prediction in predictions:
        box = prediction["box"]
        label = prediction["label"]
        score = prediction["score"]
        xmin, ymin, xmax, ymax = box.values()
        draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
        draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="white")
    return image, predictions

demo= gr.Interface(fn=detect_zeroshot, 
             inputs=[gr.Textbox(label='comma separated labels'), gr.Image(type="pil")],
             outputs=[gr.Image(type="pil"), gr.JSON()])

demo.launch()